# Analysing an Interest Rate Swap - Customising Schedules

You can override the schedule dates if you need to.

In [1]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.market.curves import *

###################################################################
# FINANCEPY BETA Version 0.180 - This build: 22 Sep 2020 at 19:38 #
#     This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy #
###################################################################



In [2]:
valuation_date = Date(20, 10, 2015)

In [3]:
discount_curve = DiscountCurveFlat(valuation_date, 
                                     0.01,
                                     FrequencyTypes.CONTINUOUS,
                                     DayCountTypes.ACT_365F)

In [4]:
index_curve = DiscountCurveFlat(valuation_date, 
                                  0.02,
                                  FrequencyTypes.CONTINUOUS,
                                  DayCountTypes.ACT_365F)

## Creating the Swap

In [5]:
settlement_date = valuation_date.add_weekdays(5)

In [6]:
settlement_date

TUE 27 OCT 2015

In [7]:
maturity_date = settlement_date.add_tenor("10Y")

We begin by setting the market conventions we wish to use.

In [8]:
swapCalendarType = CalendarTypes.UNITED_STATES
bus_day_adjust_type = BusDayAdjustTypes.MODIFIED_FOLLOWING
date_gen_rule_type = DateGenRuleTypes.FORWARD

We then define the Fixed Leg

In [9]:
fixed_coupon = 0.025
fixedFreqType = FrequencyTypes.SEMI_ANNUAL
fixed_day_count_type = DayCountTypes.ACT_360

And then we define the Floating Leg

In [10]:
float_spread = 0.004
floatFreqType = FrequencyTypes.QUARTERLY
float_day_count_type = DayCountTypes.ACT_360

We need to specify the size and direction of the swap.

In [12]:
swapType = SwapTypes.PAY
notional = 10 * ONE_MILLION

And then we define when the swap begins and ends.

We can now create the swap object.

In [16]:
swap = IborSwap(settlement_date,
                    maturity_date,
                    swapType,
                    fixed_coupon,
                    fixedFreqType,
                    fixed_day_count_type,
                    notional,
                    float_spread,
                    floatFreqType,
                    float_day_count_type,
                    swapCalendarType,
                    bus_day_adjust_type,
                    date_gen_rule_type)

In [ ]:
swap.value(valuation_date, discount_curve, index_curve)

In [ ]:
swap.print_fixed_leg_pv()

In [ ]:
swap.print_float_leg_pv()

## Customising the Schedules

Suppose I want to customise the schedules in a way that is not possible using the IborSwap constructor. This can easily be done by overriding the internal variable that stores the schedule of dates. For example, suppose I wish to make the fixed leg have odd payments as follows.

In [177]:
fixedSchedule = [valuation_date, Date(15, 11, 2015), Date(15,11, 2020), Date(15,11,2025)]

The first date is assumed to be the previous coupon date. We can set the schedule easily.

In [182]:
swap._adjustedFixedDates = fixedSchedule

Now I redo the valuation

In [183]:
swap.value(valuation_date, discount_curve, index_curve)

-78021.22225541156

We can then check to see how the valuation is arrived at.

In [184]:
swap.print_fixed_leg_pv()

START DATE: TUE 27 OCT 2015
MATURITY DATE: MON 27 OCT 2025
COUPON (%): 2.5
FIXED LEG FREQUENCY: FrequencyTypes.SEMI_ANNUAL
FIXED LEG DAY COUNT: DayCountTypes.ACT_360
VALUATION DATE TUE 20 OCT 2015
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
TUE 20 OCT 2015          -            -   1.00000000            -            -
SUN 15 NOV 2015  0.0722222     18055.56   0.99928792     18042.70     18042.70
SUN 15 NOV 2020  5.0750000   1268750.00   0.95049999   1205946.87   1223989.57
SAT 15 NOV 2025  5.0722222   1268055.56   0.90411879   1146472.86   2370462.42


Clearly the coupons have been adjusted. We do not the ability to override the flow amounts.

The floating leg is not affected.

Copyright (c) 2020 Dominic O'Kane